<a href="https://colab.research.google.com/github/Rakshakvinayakpichuka/FMML-LAB-22B21A45A9/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

Increase Percentage of Validation Set:

Positive Impact on Model Evaluation: Increasing the percentage of data allocated to the validation set can have a positive impact on model evaluation, particularly if your original validation set was very small. A larger validation set allows for better estimation of the model's performance on unseen data, which can result in a more reliable estimate of the model's generalization performance.

Potential for Overfitting on Training Data: However, if you increase the validation set size significantly while keeping the training set size constant, your model might have less training data. This can potentially lead to overfitting on the training data, as the model has less data to learn from.

Reduced Training Data: A larger validation set means a smaller training set. This might be problematic if you have limited data, as reducing the training data can result in a model that doesn't generalize well.

Decrease Percentage of Validation Set:

Negative Impact on Model Evaluation: Reducing the percentage of data allocated to the validation set can have a negative impact on model evaluation. With a smaller validation set, your estimate of the model's performance on unseen data might be less reliable. There's a higher chance of variance in your evaluation metrics, and it might not accurately represent the model's true generalization performance.

More Training Data: On the positive side, reducing the validation set size means you have more data available for training. This can be beneficial if you have a limited amount of data and want to make the most of it for model training.

In practice, the choice of the percentage of data allocated to the validation set is often a trade-off. It depends on the size of your overall dataset, the specific problem you're trying to solve, and the goals of your model. Cross-validation techniques, such as k-fold cross-validation, can be used to mitigate some of the issues associated with changing the validation set size, as they provide multiple splits of the data into training and validation sets to get a more robust estimate of model performance.



Large Training Set, Small Validation Set:

Overfitting Risk: With a large training set and a small validation set, there's a higher risk of overfitting. The model may perform exceptionally well on the training data but poorly on unseen data (including the test set). This is because the model has more opportunities to memorize the training data rather than learning general patterns.

Unreliable Validation Estimate: The accuracy on the validation set may not be a good indicator of the model's true generalization performance. It may overestimate the model's abilities.

Balanced Training and Validation Sets:

Better Generalization Estimate: When the training and validation sets are of similar sizes, it's easier to get a more reliable estimate of the model's generalization performance. The validation set is more representative of unseen data, and the model is less likely to overfit.

Predictive Accuracy for Test Set: The accuracy on the validation set is a better predictor of the accuracy on the test set in this scenario. If the model performs well on the validation set, it's more likely to perform well on the test set.

Small Training Set, Large Validation Set:

Underfitting Risk: With a small training set and a large validation set, the model may underfit because it doesn't have enough training data to learn the underlying patterns. This can lead to poor performance on both the validation and test sets.

Unreliable Validation Estimate: Similar to the case with a small validation set, a small training set can result in an unreliable estimate of the model's generalization performance.



Common Split Ratios:

A common split ratio is 70-80% of the data for training and the remaining 20-30% for validation. This ratio is often a good starting point for medium-sized datasets.

For very large datasets, you might allocate an even smaller percentage, such as 90% for training and 10% for validation.

For very small datasets, you might consider techniques like k-fold cross-validation to make the most of your limited data.

Consider the Size of Your Total Dataset:

If you have a small dataset (e.g., a few hundred samples), you should lean towards a smaller validation set to ensure that you have enough data for training. A 80-20 or 90-10 train-validation split might be reasonable.

With a larger dataset (e.g., tens of thousands of samples), you can allocate a larger portion to the validation set without compromising the amount of data available for training.

Use Cross-Validation for Small Datasets:

If your dataset is small, consider using k-fold cross-validation. This technique divides the data into k subsets (folds) and iteratively uses each fold as the validation set while training on the rest. This helps you make efficient use of your limited data and provides more reliable performance estimates.
Domain and Problem-Specific Considerations:

Some domains or specific problems may require adjustments to the standard split ratios. For example, in time series data, you may want to reserve a more recent portion of the data for validation to mimic real-world deployment scenarios.
Iterate and Experiment:

It's often a good practice to iterate and experiment with different split ratios and validation strategies. You can observe how your model's performance varies with different ratios and choose the one that best balances training and validation needs.


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Doetion accuracy across multiple splits give more consistent results?
2. Does it give more accurates averaging the valida estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Reduced Variance: When you perform a single train-validation split, the performance estimate can be highly dependent on the particular random split you choose. By performing multiple splits and averaging the results, you reduce the impact of this randomness. This helps you obtain a more stable and less variable estimate of your model's performance.

Better Representation of Data: Cross-validation ensures that every data point is used for validation exactly once (in one of the folds) and for training in other folds. This maximizes the use of your available data for both training and validation, providing a more comprehensive assessment of how well your model generalizes.

Identifying Overfitting: Cross-validation can help you detect overfitting more effectively. If a model performs exceptionally well on one training-validation split but poorly on another, it suggests that the model might be overfitting to the specific data in the first split.

Model Selection and Hyperparameter Tuning: Cross-validation is commonly used for model selection and hyperparameter tuning. By comparing the performance of different models or hyperparameters across multiple cross-validation folds, you can make more informed decisions about which model or settings to choose.

Common types of cross-validation include k-fold cross-validation and stratified k-fold cross-validation, where the dataset is divided into k subsets (folds), and the model is trained and validated k times, with each fold serving as the validation set exactly once.



Cross-Validation (CV):

Cross-validation estimates how well your model is likely to generalize to unseen data, similar to what the test set measures.

CV provides a more robust estimate of model performance compared to a single train-validation split by averaging performance across multiple validation sets (folds). This reduces the impact of random variability in the data split.

CV is useful for model selection, hyperparameter tuning, and assessing the overall quality of your model. It helps you choose the best-performing model or parameter settings.

The performance metrics (e.g., accuracy, F1 score) obtained from cross-validation are estimates of how well your model will perform on unseen data but are not the same as test set performance.

Test Set Accuracy:

The test set represents a completely independent and unseen dataset. It is used to evaluate the final, chosen model after all model development and validation steps.

Test set accuracy provides a direct measure of how well your model is expected to perform in real-world applications or on new, unseen data.

Test set accuracy is typically considered the gold standard for estimating a model's performance because it truly assesses how the model generalizes to unseen data.



Early Stopping:

Increasing the number of training iterations can sometimes lead to overfitting. Overfitting occurs when the model becomes too complex and starts fitting the noise in the training data rather than the underlying patterns. This can result in high training accuracy but poor generalization to unseen data.

To mitigate overfitting and obtain a better estimate of performance, it's common to use techniques like "early stopping." Early stopping involves monitoring the model's performance on a validation set during training. When the validation performance starts to degrade (e.g., validation loss increases), training is stopped. This prevents the model from overfitting and can lead to a better estimate of how well the model will perform on unseen data.

Optimal Epochs:

The optimal number of iterations (epochs) can vary depending on the dataset, model architecture, and problem. There is usually a "sweet spot" where the model achieves the best balance between underfitting and overfitting.

Too few iterations might result in underfitting, where the model has not learned enough from the data, leading to poor performance.

Too many iterations can lead to overfitting, as mentioned earlier, where the model becomes too specific to the training data.

A better estimate of performance is often achieved by finding the right number of iterations through experimentation, such as hyperparameter tuning.

Regularization Techniques:

Regularization techniques like dropout, weight decay, and batch normalization can help mitigate overfitting and allow the model to train for more epochs without degradation in performance.
Learning Rate Scheduling:

Adaptive learning rate schedules (e.g., learning rate annealing or learning rate decay) can be used to adjust the learning rate during training. This can help the model converge more effectively and reduce the risk of overfitting.


Overfitting Risk: With a very small training dataset, there is a higher risk of overfitting. The model might memorize the training data rather than learning the underlying patterns. Increasing the number of iterations without other measures can exacerbate overfitting, as the model has more opportunities to fit the noise in the data.

Regularization: If you have a small training dataset, it's crucial to incorporate regularization techniques like dropout, weight decay, or batch normalization. These techniques can help the model generalize better and reduce the risk of overfitting, allowing you to train for more iterations.

Data Augmentation: Data augmentation techniques can artificially increase the effective size of your training dataset by creating variations of existing data samples. This can help the model generalize better even with a small training dataset.

Transfer Learning: In cases where you have a very small dataset, transfer learning, using a pre-trained model as a starting point and fine-tuning it on your small dataset, can be a valuable approach.

For a Very Small Validation Dataset:

Statistical Variability: With a very small validation dataset, the performance estimate can be highly variable and less reliable. Increasing the number of iterations may provide a more stable estimate of the model's performance, but it doesn't fundamentally address the issue of limited data for validation.

Cross-Validation: If you have a small validation dataset, consider using cross-validation with multiple folds to obtain a more robust estimate of your model's performance. Cross-validation involves training and validating the model multiple times with different subsets of your data.

Data Splitting Considerations: If you are limited by a very small dataset and cannot obtain more data, consider revisiting your data splitting strategy. For example, you might allocate a larger portion of the data to the training set and use k-fold cross-validation to assess performance more reliably.
